# Specify you data path

Here you should specify your data path as well as the directory needed for result of RFP.

`DATA_DIR` should be addressed to you `.wav` files.

You also need to have access to `Google Cloud` and `TPU` to run the following code. Remeber to move the `vocab files` needed to your bucket in the addresses mentioned in the code.

In [ ]:
DATA_DIR = "..."
RESULT_DIR = "..."

# Cloning the repository

In [ ]:
%cd /content/
!git clone https://github.com/AmirAbaskohi/Automatic-Speech-recognition-for-Speech-Assessment-of-Persian-Preschool-Children.git

# Data preparing with RFP

Using the following cell, you can prepare data for training using RFP. Remember that you have to specify `DATA_DIR` and `RESULT_DIR` before running the cell below.

In [ ]:
%cd /content/Automatic-Speech-recognition-for-Speech-Assessment-of-Persian-Preschool-Children/Pretrain

!pip3 install -r requirements.txt
!python3 pitch_changer.py $DATA_DIR $RESULT_DIR

%cd /content/

#Pretrain model

First specify your TPU and your BUCKET that data exists. Then use the next cell to train your data.

In [ ]:
TPU_NAME = "..."
BUCKET_NAME = "..."
DATA_PATH = "..."

In [5]:
%cd /content/Automatic-Speech-recognition-for-Speech-Assessment-of-Persian-Preschool-Children/Models/Wav2Vec 2.0/bin

!pip3 install -r /content/Automatic-Speech-recognition-for-Speech-Assessment-of-Persian-Preschool-Children/Models/requirements.txt
!TPU_NAME=$TPU_NAME BUCKET_NAME=$BUCKET_NAME DATA_PATH=$DATA_PATH python3 train.py

%cd /content/

# Validate model

In [ ]:
%cd /content/Automatic-Speech-recognition-for-Speech-Assessment-of-Persian-Preschool-Children/Models/Wav2Vec 2.0
!wget https://www.openslr.org/resources/12/test-clean.tar.gz
!tar -xf test-clean.tar.gz

In [ ]:
ADDRESS_OF_PRETRAINED_MODEL = "..."

In [ ]:
import tensorflow as tf
from layers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(ADDRESS_OF_PRETRAINED_MODEL)

In [ ]:
def tf_forward(speech):
  tf_out = model(speech, training=False)
  return tf.squeeze(tf.argmax(tf_out, axis=-1))

In [ ]:
import soundfile as sf
import os

REQUIRED_SAMPLE_RATE = 16000
SPLIT = "test-clean"

def read_txt_file(f):
  with open(f, "r") as f:
    samples = f.read().split("\n")
    samples = {s.split()[0]: " ".join(s.split()[1:]) for s in samples if len(s.split()) > 2}
  return samples

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".flac")]
  return {file_id: audio}

In [ ]:
def fetch_sound_text_mapping():
  flac_files = tf.io.gfile.glob(f"LibriSpeech/{SPLIT}/*/*/*.flac")
  txt_files = tf.io.gfile.glob(f"LibriSpeech/{SPLIT}/*/*/*.txt")

  txt_samples = {}
  for f in txt_files:
    txt_samples.update(read_txt_file(f))

  speech_samples = {}
  for f in flac_files:
    speech_samples.update(read_flac_file(f))

  file_ids = set(speech_samples.keys()) & set(txt_samples.keys())
  print(f"{len(file_ids)} files are found in LibriSpeech/{SPLIT}")
  samples = [(speech_samples[file_id], txt_samples[file_id]) for file_id in file_ids]
  return samples

In [ ]:
samples = fetch_sound_text_mapping()

In [ ]:
from layers import Wav2Vec2Processor

tokenizer = Wav2Vec2Processor(is_tokenizer=True)
processor = Wav2Vec2Processor(is_tokenizer=False)

In [ ]:
AUDIO_MAXLEN, LABEL_MAXLEN = 246000, 256
DO_PADDING = False

def preprocess_text(text):
  label = tokenizer(text)
  label = tf.constant(label, dtype=tf.int32)[None]
  if DO_PADDING:
    label = label[:, :LABEL_MAXLEN]
    padding = tf.zeros((label.shape[0], LABEL_MAXLEN - label.shape[1]), dtype=label.dtype)
    label = tf.concat([label, padding], axis=-1)
  return label

def preprocess_speech(audio):
  audio = tf.constant(audio, dtype=tf.float32)
  audio = processor(audio)[None]
  if DO_PADDING:
    audio = audio[:, :AUDIO_MAXLEN]
    padding = tf.zeros((audio.shape[0], AUDIO_MAXLEN - audio.shape[1]), dtype=audio.dtype)
    audio = tf.concat([audio, padding], axis=-1)
  return audio

In [ ]:
def inputs_generator():
  for speech, text in samples:
    yield preprocess_speech(speech), preprocess_text(text)

output_signature = (
    tf.TensorSpec(shape=(None),  dtype=tf.float32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
)
dataset = tf.data.Dataset.from_generator(inputs_generator, output_signature=output_signature)

In [ ]:
from tqdm.auto import tqdm

def infer_librispeech(dataset: tf.data.Dataset, num_batches: int = None):
  predictions, labels = [], []
  for batch in tqdm(dataset, total=num_batches, desc="LibriSpeech Inference ... "):
    speech, label = batch
    tf_out = tf_forward(speech)
    predictions.append(tokenizer.decode(tf_out.numpy().tolist(), group_tokens=True))
    labels.append(tokenizer.decode(label.numpy().squeeze().tolist(), group_tokens=False))
  return predictions, labels

In [ ]:
predictions, labels = infer_librispeech(dataset, num_batches=2618)

In [ ]:
from datasets import load_metric

wer = load_metric("wer")
wer.compute(references=labels, predictions=predictions)

In addition to the following code I have, you can use the `evaluation.py` in the `bin` directory.